In [39]:

import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np


page = urlopen("https://en.wikipedia.org/wiki/List_of_UFC_events")
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
table = soup.find("table", {"id": "Past_events"})
tbody = table.find("tbody")

rows = tbody.find_all("tr")
rows = rows[1:]

data = []

for row in rows:
  tds = row.find_all("td")
  row_data = []

  if (len(tds) == 7) and ((tds[0].get_text(strip=True)) != "—"):
    row_data = row_data + [tds[0].get_text(strip=True)]
    row_data = row_data + [tds[1].get_text(strip=True)]
    row_data = row_data + [tds[3].get_text(strip=True)]
    if tds[3].find('a', href=True) != None:
      link = tds[3].find('a', href=True)
      href = link.get('href')
      row_data = row_data + [href]
    else:
      row_data = row_data + [None]
    row_data = row_data + [tds[5].get_text(strip=True)]
    data = data + [row_data]

  if (len(tds) == 6) and ((tds[0].get_text(strip=True)) != "—"):
    row_data = row_data + [tds[0].get_text(strip=True)]
    row_data = row_data + [tds[1].get_text(strip=True)]
    row_data = row_data + [tds[3].get_text(strip=True)]
    if tds[3].find('a', href=True) != None:
      link = tds[3].find('a', href=True)
      href = link.get('href')
      row_data = row_data + [href]
    else:
      row_data = row_data + [None]
    row_data = row_data + [tds[4].get_text(strip=True)]
    data = data + [row_data]

  if (len(tds) == 5) and ((tds[0].get_text(strip=True)) != "—"):
    row_data = row_data + [tds[0].get_text(strip=True)]
    row_data = row_data + [tds[1].get_text(strip=True)]
    row_data = row_data + [None]
    row_data = row_data + [None]
    row_data = row_data + [tds[3].get_text(strip=True)]
    data = data + [row_data]

  if (len(tds) == 4) and ((tds[0].get_text(strip=True)) != "—"):
    row_data = row_data + [tds[0].get_text(strip=True)]
    row_data = row_data + [tds[1].get_text(strip=True)]
    row_data = row_data + [None]
    row_data = row_data + [None]
    row_data = row_data + ["—"]
    data = data + [row_data]

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
df = pd.DataFrame(data, columns=['event_id', 'event_name', "venue", "venue_link", "attendance"])
df["event_id"] = df["event_id"].astype(int)
df['attendance'] = df['attendance'].replace('—', np.nan)
df['attendance'] = df['attendance'].replace({',': ''}, regex=True)
df['attendance'] = df['attendance'].astype(str)
df['attendance'] = df['attendance'].apply(lambda x: np.nan if x[0].isalpha() else x)
df['venue'] = df['venue'].ffill()
df['venue_link'] = df['venue_link'].ffill()
df.loc[df['event_id'] == 465, 'venue_link'] = None
df.loc[df['event_id'] == 428, 'venue_link'] = None
df.loc[df['event_id'] == 392, 'venue_link'] = None
df["venue_link"] = "https://en.m.wikipedia.org/" + df["venue_link"]
df['venue'] = df['venue'].astype(str)
df = df.fillna('')
df['venue_link'] = df['venue_link'].astype(str)
df['event_name'] = df['event_name'].astype(str)
df['event_id'] = df['event_id'].astype(int)
df.head()

,event_id,event_name,venue,venue_link,attendance
0,716,UFC on ESPN: Covington vs. Buckley,Amalie Arena,https://en.m.wikipedia.org//wiki/Amalie_Arena,18625
1,715,UFC 310: Pantoja vs. Asakura,T-Mobile Arena,https://en.m.wikipedia.org//wiki/T-Mobile_Arena,18648
2,714,UFC Fight Night: Yan vs. Figueiredo,Galaxy Arena,https://en.m.wikipedia.org//wiki/Galaxy_Macau#...,12615
3,713,UFC 309: Jones vs. Miocic,Madison Square Garden,https://en.m.wikipedia.org//wiki/Madison_Squar...,20200
4,712,UFC Fight Night: Magny vs. Prates,UFC Apex,https://en.m.wikipedia.org//wiki/UFC_Apex,


In [47]:
import requests
from bs4 import BeautifulSoup

links = df['venue_link'].tolist()
new_links = []
counter = 0

for link in links:
  if link == '':
    new_links = new_links + [None]
    print(counter, None)
  else:
    response = requests.get(link)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    link = soup.find('a', class_='external text')
    link = (link.get('href'))
    if (link[0:11]) == "https://geo":
      print(counter, link)
      page = urlopen(link)
      html = page.read().decode("utf-8")
      soup = BeautifulSoup(html, "html.parser")
      latitude = soup.find('span', class_='latitude')
      longitude = soup.find('span', class_='longitude')
      new_links = new_links + [[latitude.get_text(), longitude.get_text()]]
    else:
      print(counter, link)
      new_links = new_links + [link]
    counter = counter + 1

0 https://geohack.toolforge.org/geohack.php?pagename=Amalie_Arena&params=27_56_34_N_82_27_7_W_type:landmark_scale:2000
1 https://geohack.toolforge.org/geohack.php?pagename=T-Mobile_Arena&params=36_06_10_N_115_10_42_W_type:landmark_scale:2000
2 http://www.galaxymacau.com/en
3 https://geohack.toolforge.org/geohack.php?pagename=Madison_Square_Garden&params=40_45_2_N_73_59_37_W_type:landmark_scale:1000
4 https://geohack.toolforge.org/geohack.php?pagename=UFC_Apex&params=36_4_4.8_N_115_13_40.8_W_type:landmark_region:US-NV
5 https://geohack.toolforge.org/geohack.php?pagename=Rogers_Place&params=53_32_49_N_113_29_52_W_type:landmark_region:CA-AB
6 https://geohack.toolforge.org/geohack.php?pagename=Etihad_Arena&params=24.4603_N_54.6040_E_type:landmark
7 https://geohack.toolforge.org/geohack.php?pagename=UFC_Apex&params=36_4_4.8_N_115_13_40.8_W_type:landmark_region:US-NV
8 https://geohack.toolforge.org/geohack.php?pagename=UFC_Apex&params=36_4_4.8_N_115_13_40.8_W_type:landmark_region:US-NV
9 htt

In [48]:
df["venue_link"] = new_links
df['venue_link'] = df.apply(lambda row: row['venue'] if isinstance(row['venue_link'], str) or row['venue_link'] is None  else row['venue_link'], axis=1)

In [49]:
links = df['venue_link'].tolist()
print(links)

[['27.942778', '-82.451944'], ['36.102778', '-115.178333'], 'Galaxy Arena', ['40.750556', '-73.993611'], ['36.068', '-115.228'], ['53.546944', '-113.497778'], ['24.4603', '54.604'], ['36.068', '-115.228'], ['36.068', '-115.228'], ['40.768333', '-111.901111'], ['48.838611', '2.378611'], ['36.120556', '-115.161389'], ['36.068', '-115.228'], ['36.068', '-115.228'], ['-31.9483', '115.8519'], ['36.068', '-115.228'], ['24.4603', '54.604'], ['53.486389', '-2.199722'], ['36.068', '-115.228'], 'Ball Arena', ['36.102778', '-115.178333'], 'Kingdom Arena', ['36.068', '-115.228'], ['38.2575', '-85.753889'], ['40.733611', '-74.171111'], ['36.068', '-115.228'], ['38.626667', '-90.2025'], ['-22.975278', '-43.390278'], ['36.068', '-115.228'], ['36.102778', '-115.178333'], ['36.068', '-115.228'], 'Boardwalk Hall', ['36.068', '-115.228'], ['36.068', '-115.228'], ['25.781389', '-80.188056'], ['36.068', '-115.228'], ['19.496309', '-99.175429'], ['33.807778', '-117.876667'], ['36.068', '-115.228'], ['36.068

In [50]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderUnavailable, GeocoderTimedOut
import time
import requests

coordinate_list = []

def get_lat_long(location_name, delay=2):
    geolocator = Nominatim(user_agent="my_app")
    while True:  # Infinite loop to keep retrying forever
        try:
            location = geolocator.geocode(location_name)
            if location:
                return location.latitude, location.longitude
            else:
                print(f"Location '{location_name}' not found.")
                return [0, 0]
        except (GeocoderUnavailable, GeocoderTimedOut, requests.exceptions.ReadTimeout) as e:
            print(f"Error: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)  # Wait before retrying
        except Exception as e:
            print(f"Unexpected error: {e}")
            return None  # Stop if an unexpected error occurs

# Assuming 'links' is a list of locations you want to process
for link in links:
    if isinstance(link, list):
        coordinate_list += [link]  # Add list of coordinates to coordinate_list
    else:
        result = get_lat_long(link)
        print(result, link)
        if result:
            coordinate_list.append(result)

print(coordinate_list)




(22.1439403, 113.5568931) Galaxy Arena
(39.748683799999995, -105.00754401780362) Ball Arena


(32.751551, -117.2071972) Kingdom Arena
(41.75195805, -72.69158289823056) Boardwalk Hall


Error: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=The+Theater+at+Virgin+Hotels&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")). Retrying in 2 seconds...
Location 'The Theater at Virgin Hotels' not found.
[0, 0] The Theater at Virgin Hotels
Location 'Gila River Arena' not found.
[0, 0] Gila River Arena
Location 'Ginásio Nilson Nelson' not found.
[0, 0] Ginásio Nilson Nelson
(35.1923402, 129.06070465) Sajik Arena
(55.701579100000004, 37.6448362526219) CSKA Arena


Error: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Shenzhen+Universiade+Sports+Centre&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")). Retrying in 2 seconds...
(22.69864365, 114.21105502765981) Shenzhen Universiade Sports Centre
(59.2936201, 18.083213182712974) Avicii Arena
(43.153747949999996, -77.61082627098224) Blue Cross Arena
(45.296906899999996, -75.92689732572379) Canadian Tire Centre
(50.104754650000004, 14.493621810111105) O2 Arena
(-3.8048605, -38.52461420580107) Centro de Formação Olímpica do Nordeste
(33.95822585, -118.34198075000555) The Forum
(-34.9079418, 138.57380042679154) Adelaide Entertainment Centre
(39.748683799999995, -105.00754401780362) Pepsi Center


Location 'Adirondack Bank Center' not found.
[0, 0] Adirondack Bank Center
(41.75195805, -72.69158289823056) Boardwalk Hall
Location 'Gila River Arena' not found.
[0, 0] Gila River Arena
(-1.3826760500000002, -48.439456709962656) Arena Guilherme Paraense
(35.8951196, 139.63075300618382) Saitama Super Arena
(51.8832963, 4.4883655) Rotterdam Ahoy
(59.2936201, 18.083213182712974) Ericsson Globe


...

Location 'Tokyo Bay NK Hall' not found.
[0, 0] Tokyo Bay NK Hall
(30.22856595, -93.22142555923101) Lake Charles Civic Center
Location 'Five Seasons Events Center' not found.
[0, 0] Five Seasons Events Center
Location 'Boutwell Memorial Auditorium' not found.
[0, 0] Boutwell Memorial Auditorium


Error: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Casino+Magic+Bay+St.+Louis&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")). Retrying in 2 seconds...
Location 'Casino Magic Bay St. Louis' not found.
[0, 0] Casino Magic Bay St. Louis
(30.039359949999998, -90.24057621223363) Pontchartrain Center
(30.039359949999998, -90.24057621223363) Pontchartrain Center


Error: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Casino+Magic+Bay+St.+Louis&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")). Retrying in 2 seconds...
Location 'Casino Magic Bay St. Louis' not found.
[0, 0] Casino Magic Bay St. Louis
Location 'Boutwell Memorial Auditorium' not found.
[0, 0] Boutwell Memorial Auditorium
(44.34259245, -69.79889863924052) Augusta Civic Center
(31.22485515, -85.39032417991697) Dothan Civic Center


(30.647044, -97.00745087686691) Fair Park Arena
(44.34259245, -69.79889863924052) Augusta Civic Center


(30.647044, -97.00745087686691) Fair Park Arena
Location 'Casper Events Center' not found.
[0, 0] Casper Events Center
(39.03986125, -94.35749672753282) Independence Arena
(35.2190348, -80.82924452265625) Grady Cole Center
[['27.942778', '-82.451944'], ['36.102778', '-115.178333'], (22.1439403, 113.5568931), ['40.750556', '-73.993611'], ['36.068', '-115.228'], ['53.546944', '-113.497778'], ['24.4603', '54.604'], ['36.068', '-115.228'], ['36.068', '-115.228'], ['40.768333', '-111.901111'], ['48.838611', '2.378611'], ['36.120556', '-115.161389'], ['36.068', '-115.228'], ['36.068', '-115.228'], ['-31.9483', '115.8519'], ['36.068', '-115.228'], ['24.4603', '54.604'], ['53.486389', '-2.199722'], ['36.068', '-115.228'], (39.748683799999995, -105.00754401780362), ['36.102778', '-115.178333'], (32.751551, -117.2071972), ['36.068', '-115.228'], ['38.2575', '-85.753889'], ['40.733611', '-74.171111'], ['36.068', '-115.228'], ['38.626667', '-90.2025'], ['-22.975278', '-43.390278'], ['36.068', '-11

In [51]:
df['venue_link'] = coordinate_list
df['venue_link'] = df['venue_link'].apply(tuple)
df['latitude'] = df['venue_link'].apply(lambda x: float(x[0]))
df['longitude'] = df['venue_link'].apply(lambda x: float(x[1]))

# Casper Events Center
df.loc[df['event_id'] == 6, 'latitude'] = 42.8683
df.loc[df['event_id'] == 6, 'longitude'] = -106.3328

# Boutwell Memorial Auditorium
df.loc[df['event_id'] == 16, 'latitude'] = 33.52159
df.loc[df['event_id'] == 16, 'longitude'] = -86.81101
df.loc[df['event_id'] == 24, 'latitude'] = 33.52159
df.loc[df['event_id'] == 24, 'longitude'] = -86.81101

# Casino Magic Bay St. Louis
df.loc[df['event_id'] == 17, 'latitude'] = 30.3348
df.loc[df['event_id'] == 17, 'longitude'] = -89.3538
df.loc[df['event_id'] == 23, 'latitude'] = 30.3348
df.loc[df['event_id'] == 23, 'longitude'] = -89.3538

# Five Seasons Events Center
df.loc[df['event_id'] == 25, 'latitude'] = 41.9802
df.loc[df['event_id'] == 25, 'longitude'] = -91.6675
df.loc[df['event_id'] == 30, 'latitude'] = 41.9802
df.loc[df['event_id'] == 30, 'longitude'] = -91.6675

# Tokyo Bay NK Hall
df.loc[df['event_id'] == 27, 'latitude'] = 35.6227141758
df.loc[df['event_id'] == 27, 'longitude'] = 139.871809846

# Metro Radio Arena
df.loc[df['event_id'] == 103, 'latitude'] = 54.9639
df.loc[df['event_id'] == 103, 'longitude'] = -1.623333

# Broomfield Event Center
df.loc[df['event_id'] == 107, 'latitude'] = 39.920540
df.loc[df['event_id'] == 107, 'longitude'] = -105.086647

# Revel Casino Hotel
df.loc[df['event_id'] == 208, 'latitude'] = 39.3628
df.loc[df['event_id'] == 208, 'longitude'] = -74.4158
df.loc[df['event_id'] == 282, 'latitude'] = 39.3628
df.loc[df['event_id'] == 282, 'longitude'] = -74.4158

# Gold Coast Convention and Exhibition Centre
df.loc[df['event_id'] == 222, 'latitude'] = -28.0279
df.loc[df['event_id'] == 222, 'longitude'] = 153.4286

# Ginasio Nilson Nelson
df.loc[df['event_id'] == 290, 'latitude'] = -15.7831
df.loc[df['event_id'] == 290, 'longitude'] = -47.9032
df.loc[df['event_id'] == 373, 'latitude'] = -15.7831
df.loc[df['event_id'] == 373, 'longitude'] = -47.9032
df.loc[df['event_id'] == 513, 'latitude'] = -15.7831
df.loc[df['event_id'] == 513, 'longitude'] = -47.9032

# The SSE Arena
df.loc[df['event_id'] == 378, 'latitude'] = 54.6036
df.loc[df['event_id'] == 378, 'longitude'] = -5.9142

# Gila River Arena
df.loc[df['event_id'] == 435, 'latitude'] = 33.5321
df.loc[df['event_id'] == 435, 'longitude'] = -112.2612
df.loc[df['event_id'] == 566, 'latitude'] = 33.5321
df.loc[df['event_id'] == 566, 'longitude'] = -112.2612

# Adirondack Bank Center
df.loc[df['event_id'] == 440, 'latitude'] = 43.1050
df.loc[df['event_id'] == 440, 'longitude'] = -75.2334

# The Theater at Virgin Hotels
df.loc[df['event_id'] == 639, 'latitude'] = 36.1093
df.loc[df['event_id'] == 639, 'longitude'] = -115.1531

pd.set_option('display.max_rows', None)
print(df)



     event_id                                         event_name                                        venue                                  venue_link attendance   latitude   longitude
0         716                 UFC on ESPN: Covington vs. Buckley                                 Amalie Arena                     (27.942778, -82.451944)      18625  27.942778  -82.451944
1         715                       UFC 310: Pantoja vs. Asakura                               T-Mobile Arena                    (36.102778, -115.178333)      18648  36.102778 -115.178333
2         714                UFC Fight Night: Yan vs. Figueiredo                                 Galaxy Arena                   (22.1439403, 113.5568931)      12615  22.143940  113.556893
3         713                          UFC 309: Jones vs. Miocic                        Madison Square Garden                     (40.750556, -73.993611)      20200  40.750556  -73.993611
4         712                  UFC Fight Night: Magny vs. Pr

In [54]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
import concurrent.futures
import time
import random
import pandas as pd

def fetch_event_info(event_counter):
    url = "http://ufcstats.com/statistics/events/completed?page=all"

    while True:
        try:
            page = urlopen(url)
            html = page.read().decode("utf-8")
            soup = BeautifulSoup(html, "html.parser")

            events = soup.find_all("tr", class_="b-statistics__table-row")[event_counter]
            event_name = events.find("a").text.strip()
            event_link = events.find("a").get("href")
            date = events.find("span").text.strip()
            location = events.find_all("td")[-1].text.strip()

            date_parts = date.split()
            year = date_parts[2]
            day = date_parts[1][:-1]
            month = date_parts[0]
            month_map = {
                "January": "01", "February": "02", "March": "03",
                "April": "04", "May": "05", "June": "06",
                "July": "07", "August": "08", "September": "09",
                "October": "10", "November": "11", "December": "12"
            }
            month = month_map.get(month, "00")
            date = f"{year}-{month}-{day}"

            location_parts = location.split(", ")
            if len(location_parts) == 2:
                city = location_parts[0]
                region = location_parts[0]
                nation = location_parts[1]
            elif len(location_parts) == 3:
                city = location_parts[0]
                region = location_parts[1]
                nation = location_parts[2]

            return (event_counter, [event_counter, event_name, date, city, region, nation, event_link])

        except Exception as e:
            print(f"Error fetching info for event {event_counter}: {e}")
            time.sleep(random.uniform(2, 5))

url = "http://ufcstats.com/statistics/events/completed?page=all"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

event_len = len(soup.find_all("tr", class_="b-statistics__table-row"))
all_event_info = [None] * (event_len - 2)  # Preallocate the list

with open('events.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    field = ["event_id", "event_name", "date", "city", "region", "nation", "event_link"]
    writer.writerow(field)

    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        futures = {executor.submit(fetch_event_info, i): i for i in range(2, event_len)}

        for future in concurrent.futures.as_completed(futures):
            try:
                index, event_info = future.result()  # Get index and event info
                all_event_info[index] = event_info  # Place it in the correct order
                print(f"Processed event: {event_info[0]}/{event_len - 2}")
            except Exception as e:
                print(f"Error processing event: {e}")

    # Write all event info to the CSV after fetching
    for event_info in all_event_info:
        if event_info is not None:
            writer.writerow(event_info)

events = pd.read_csv("events.csv")
events
# events['event_id'] = range(1, len(events) + 1)
# events.to_csv("events.csv", index=False)

# print(f"Total events processed: {len([info for info in all_event_info if info])}")

Error fetching info for event 7: HTTP Error 429: Too Many Requests
Error fetching info for event 6: HTTP Error 429: Too Many Requests
Error fetching info for event 19: HTTP Error 429: Too Many Requests
Error fetching info for event 10: HTTP Error 429: Too Many RequestsError fetching info for event 11: HTTP Error 429: Too Many Requests

Error fetching info for event 9: HTTP Error 429: Too Many Requests
Error fetching info for event 16: HTTP Error 429: Too Many Requests
Error fetching info for event 15: HTTP Error 429: Too Many Requests
Error fetching info for event 18: HTTP Error 429: Too Many Requests
Error fetching info for event 20: HTTP Error 429: Too Many Requests
Processed event: 8/715
Processed event: 3/715
Processed event: 12/715
Processed event: 5/715
Processed event: 4/715
Processed event: 17/715
Processed event: 14/715
Processed event: 13/715
Processed event: 21/715
Processed event: 2/715
Processed event: 18/715
Processed event: 6/715
Processed event: 15/715
Processed event: 

,event_id,event_name,date,city,region,nation,event_link
0,2,UFC Fight Night: Covington vs. Buckley,2024-12-14,Tampa,Florida,USA,http://ufcstats.com/event-details/72c9c2eadfc3...
1,3,UFC 310: Pantoja vs. Asakura,2024-12-07,Las Vegas,Nevada,USA,http://ufcstats.com/event-details/ad23903ef3af...
2,4,UFC Fight Night: Yan vs. Figueiredo,2024-11-23,Macau,Macau,China,http://ufcstats.com/event-details/e955046551f8...
3,5,UFC 309: Jones vs. Miocic,2024-11-16,New York City,New York,USA,http://ufcstats.com/event-details/daff32bc96d1...
4,6,UFC Fight Night: Magny vs. Prates,2024-11-09,Las Vegas,Nevada,USA,http://ufcstats.com/event-details/585f9ffdb0cd...
5,7,UFC Fight Night: Moreno vs. Albazi,2024-11-02,Edmonton,Alberta,Canada,http://ufcstats.com/event-details/3ab0df1ddd5f...
6,8,UFC 308: Topuria vs. Holloway,2024-10-26,Abu Dhabi,Abu Dhabi,United Arab Emirates,http://ufcstats.com/event-details/221b2a3070c7...
7,9,UFC Fight Night: Hernandez vs. Pereira,2024-10-19,Las Vegas,Nevada,USA,http://ufcstats.com/event-details/6a0047ac8c99...
8,10,UFC Fight Night: Royval vs. Taira,2024-10-12,Las Vegas,Nevada,USA,http://ufcstats.com/event-details/5bdcbd8dd681...
9,11,UFC 307: Pereira vs. Rountree Jr.,2024-10-05,Salt Lake City,Utah,USA,http://ufcstats.com/event-details/eee8efec7b95...


In [73]:
df = df.iloc[:-3]

In [81]:
import pandas as pd

events = pd.read_csv('events.csv')
result = pd.merge(events, df, left_index=True, right_index=True)
result = result.drop('venue_link', axis=1)
result = result.drop('event_id_y', axis=1)
result = result.rename(columns={'event_id_x': 'event_id'})
result['event_id'] = range(len(result), 0, -1)

,event_id,event_name_x,date,city,region,nation,event_link,event_name_y,venue,attendance,latitude,longitude
0,713,UFC Fight Night: Covington vs. Buckley,2024-12-14,Tampa,Florida,USA,http://ufcstats.com/event-details/72c9c2eadfc3...,UFC on ESPN: Covington vs. Buckley,Amalie Arena,18625,27.942778,-82.451944
1,712,UFC 310: Pantoja vs. Asakura,2024-12-07,Las Vegas,Nevada,USA,http://ufcstats.com/event-details/ad23903ef3af...,UFC 310: Pantoja vs. Asakura,T-Mobile Arena,18648,36.102778,-115.178333
2,711,UFC Fight Night: Yan vs. Figueiredo,2024-11-23,Macau,Macau,China,http://ufcstats.com/event-details/e955046551f8...,UFC Fight Night: Yan vs. Figueiredo,Galaxy Arena,12615,22.143940,113.556893
3,710,UFC 309: Jones vs. Miocic,2024-11-16,New York City,New York,USA,http://ufcstats.com/event-details/daff32bc96d1...,UFC 309: Jones vs. Miocic,Madison Square Garden,20200,40.750556,-73.993611
4,709,UFC Fight Night: Magny vs. Prates,2024-11-09,Las Vegas,Nevada,USA,http://ufcstats.com/event-details/585f9ffdb0cd...,UFC Fight Night: Magny vs. Prates,UFC Apex,,36.068000,-115.228000
5,708,UFC Fight Night: Moreno vs. Albazi,2024-11-02,Edmonton,Alberta,Canada,http://ufcstats.com/event-details/3ab0df1ddd5f...,UFC Fight Night: Moreno vs. Albazi,Rogers Place,16439,53.546944,-113.497778
6,707,UFC 308: Topuria vs. Holloway,2024-10-26,Abu Dhabi,Abu Dhabi,United Arab Emirates,http://ufcstats.com/event-details/221b2a3070c7...,UFC 308: Topuria vs. Holloway,Etihad Arena,,24.460300,54.604000
7,706,UFC Fight Night: Hernandez vs. Pereira,2024-10-19,Las Vegas,Nevada,USA,http://ufcstats.com/event-details/6a0047ac8c99...,UFC Fight Night: Hernandez vs. Pereira,UFC Apex,,36.068000,-115.228000
8,705,UFC Fight Night: Royval vs. Taira,2024-10-12,Las Vegas,Nevada,USA,http://ufcstats.com/event-details/5bdcbd8dd681...,UFC Fight Night: Royval vs. Taira,UFC Apex,,36.068000,-115.228000
9,704,UFC 307: Pereira vs. Rountree Jr.,2024-10-05,Salt Lake City,Utah,USA,http://ufcstats.com/event-details/eee8efec7b95...,UFC 307: Pereira vs. Rountree Jr.,Delta Center,17487,40.768333,-111.901111


In [82]:
result.to_csv('events.csv', index=False)